In [1]:
import numpy as np
import jax.numpy as jnp
import jax
import cyipopt

from functools import partial
from typing import *
from dataclasses import dataclass, field
from jaxlie import SE3, SO3
import jax_dataclasses as jdc

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *
from sdf_world.sparse_ipopt import *

import time

In [2]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [3]:
def get_random_pose():
    pos = jnp.array([0,0,0.5]) #np.random.uniform(-1, 1, size=3)
    rot = SO3(np.random.normal(size=4)).normalize()
    return SE3.from_rotation_and_translation(rot, pos)

In [4]:
box = Box(world.vis, "box", [0.1, 0.2, 0.3], "red")
box_target = Box(world.vis, "box_target", [0.1, 0.2, 0.3], "blue")

In [5]:
box_target.set_pose(get_random_pose())

In [6]:
def qtn_err(qtn_like, qtn_d):
    rot = SO3(qtn_like).normalize()
    rot_err = rot.inverse() @ SO3(qtn_d)
    return jnp.sum(rot_err.log() ** 2)
jac_qtn_err = lambda qtn_like, qtn_d: [jax.grad(qtn_err)(qtn_like, qtn_d), np.zeros(4)]

def qtn_constr(qtn_like):
    return jnp.sum(qtn_like**2) - 1
jac_qtn_constr = jax.grad(qtn_constr, argnums=[0])

In [7]:
qtn_like = np.random.random(4)

In [8]:
frame = Frame(world.vis, "frame", length=0.3)
frame_d = Frame(world.vis, "frame_d", length=0.3)

In [9]:
# optimization + constraint
bdr1 = SparseIPOPT()
bdr1.add_variable("qtn", 4, -1., 1.)
bdr1.add_variable("pos", 3, -1., 1.)
bdr1.add_parameter("qtn_d", 4)


def debug_callback(xdict):
    qtn = xdict["qtn"]
    qtn_d = xdict["qtn_d"]
    pose = SE3.from_rotation(SO3(qtn).normalize())
    pose_d = SE3.from_rotation_and_translation(SO3(qtn_d), jnp.array([0,0,0.5]))
    box.set_pose(pose)
    frame.set_pose(pose)
    frame_d.set_pose(pose_d)
    print(f"err: {qtn_err(qtn, qtn_d)}  viol:{qtn_constr(qtn)}")
    time.sleep(0.5)
bdr1.set_debug_callback(debug_callback)

bdr1.register_fn("qtn_err", [4, 4], 1, qtn_err, jac_qtn_err)
bdr1.register_fn("qtn_constr", [4], 1, qtn_constr, jac_qtn_constr)

bdr1.set_objective("qtn_err", ["qtn", "qtn_d"])
#bdr1.set_constr("qtn_err", "qtn_err", ["qtn", "qtn_d"], 0., 0.)
bdr1.set_constr("qtn_constr", "qtn_constr", ["qtn"], 0., 0.)


In [10]:
ipopt_qtn = bdr1.build()

compiling objective ...
err: nan  viol:-1.0
compiling gradient ...
compiling constraints ...
compiling jacobian ...
oooo-------


In [11]:
qtn_d = np.random.normal(size=4)
qtn_d /= np.linalg.norm(qtn_d)

pose_d = SE3.from_rotation_and_translation(SO3(qtn_d), box_target.pose.translation())
box_target.set_pose(pose_d)
box.set_pose(SE3.identity())

In [36]:
x0 = np.hstack([np.array([1,0,0,0.]), np.zeros(3), qtn_d])
ipopt_qtn.add_option("acceptable_tol", 0.01)

#ipopt_qtn.add_option("acceptable_constr_viol_tol", 0.001)
xsol, info = ipopt_qtn.solve(x0)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       11
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

err: 9.072041511535645  viol:-0.019899964332580566
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.0720415e+00 1.99e-

In [24]:
def ec_err(ec, ec_d):
    rot = SO3.exp(ec)
    rot_err = (rot.inverse() @ SO3.exp(ec_d))
    return jnp.sum(rot_err.log() ** 2)
jac_ec_err = lambda ec, ec_d: [jax.grad(ec_err)(ec, ec_d), np.zeros(4)]

In [32]:
# optimization + constraint
bdr2 = SparseIPOPT()
bdr2.add_variable("ec", 3, -np.pi, np.pi)
bdr2.add_parameter("ec_d", 3)


def debug_callback(xdict):
    ec = xdict["ec"]
    ec_d = xdict["ec_d"]
    pose = SE3.from_rotation(SO3.exp(ec))
    pose_d = SE3.from_rotation_and_translation(SO3.exp(ec_d), jnp.array([0,0,0.5]))
    box.set_pose(pose)
    frame.set_pose(pose)
    frame_d.set_pose(pose_d)
    #print(f"err: {qtn_err(qtn, qtn_d)}  viol:{qtn_constr(qtn)}")
    time.sleep(0.5)
bdr2.set_debug_callback(debug_callback)

bdr2.register_fn("ec_err", [3, ], 1, ec_err, jac_ec_err)
#bdr1.register_fn("qtn_constr", [4], 1, qtn_constr, jac_qtn_constr)

bdr2.set_objective("ec_err", ["ec", "ec_d"])
#bdr1.set_constr("qtn_err", "qtn_err", ["qtn", "qtn_d"], 0., 0.)
#bdr2.set_constr("qtn_constr", "qtn_constr", ["qtn"], 0., 0.)


In [33]:
ipopt_ec = bdr2.build()

compiling objective ...
compiling gradient ...
no constraints


In [34]:
ec_d = SO3(qtn_d).log()
ec_init = SO3(qtn_like).log()
#pose_d = SE3.from_rotation_and_translation(SO3(qtn_d), box_target.pose.translation())
#box_target.set_pose(pose_d)
#box.set_pose(SE3.identity())

In [37]:
x0 = np.hstack([ec_init, ec_d])
ipopt_ec.add_option("acceptable_tol", 0.01)

#ipopt_qtn.add_option("acceptable_constr_viol_tol", 0.001)
xsol, info = ipopt_ec.solve(x0)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.7099586e+00 0.00e+00 3.76e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 